Imports

In [11]:
from enum import unique
from scipy.sparse.construct import rand
from sklearn.utils import shuffle
import medical_lib as ml
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")

Dataframe einlesen

In [12]:
df = pd.read_csv('filled_noStr.csv')
df = df.iloc[:, 1:]

tot = pd.read_csv('Verstorben.csv')

'Filled_noStr.csv' wird in zwei Datasets geteilt.
Df1 enthält nur die Toten. Df2 enthält nur die Lebenden.
Anschliessend werden die Daten von 94 Toten und 94 Lebenden zusammengeführt und die Daten der restlichen 20 Toten und restlichen Lebenden separat vereinigt.

Diese beiden Dataframes werden später Train und Testset sein.

In [13]:
uniquetote = tot['Pseudonym'].unique().tolist()
df1 = df[df['Pseudonym'].isin(uniquetote)] #Dataframe nur mit den Toten
df2 = df[~df['Pseudonym'].isin(uniquetote)] #Dataframe nur mit den Lebenden
uniquelebende = df2['Pseudonym'].unique().tolist() #Liste aller distinct Lebende

uniquetote1 = uniquetote[0:(len(uniquetote)-20)] #Liste aus 94 tote
uniquetote2 = uniquetote[len(uniquetote)-20: len(uniquetote)] #Liste der restlichen 20tote

uniquelebende1 = uniquelebende[0: 94] #Liste aus 94Lebende
uniquelebende2 = uniquelebende[94:len(uniquelebende)] # Liste der restlichen Lebenden

beides_Train = uniquetote1 + uniquelebende1
beides_Test = uniquetote2 + uniquelebende2

In [14]:
print(uniquelebende1)
print('')
print(uniquetote1)

[79.0, 124.0, 997.0, 1056.0, 1256.0, 1494.0, 1938.0, 2500.0, 2568.0, 2628.0, 2652.0, 2832.0, 3297.0, 3443.0, 4747.0, 5190.0, 5650.0, 6360.0, 6658.0, 7270.0, 8275.0, 8757.0, 9208.0, 9280.0, 9382.0, 9457.0, 9688.0, 11288.0, 11337.0, 11862.0, 12889.0, 13243.0, 14199.0, 14284.0, 14313.0, 16272.0, 17201.0, 17461.0, 17644.0, 18298.0, 19124.0, 21470.0, 24004.0, 25619.0, 26792.0, 28982.0, 29771.0, 30799.0, 34317.0, 36408.0, 38115.0, 39361.0, 39821.0, 45533.0, 49425.0, 49549.0, 50241.0, 50549.0, 52103.0, 53045.0, 53597.0, 54990.0, 55973.0, 57100.0, 58182.0, 58330.0, 61647.0, 65046.0, 67097.0, 68828.0, 70129.0, 71972.0, 73327.0, 74731.0, 74829.0, 77315.0, 77399.0, 78667.0, 80660.0, 84739.0, 85038.0, 88652.0, 88891.0, 89548.0, 90576.0, 90921.0, 91210.0, 93051.0, 98569.0, 100030.0, 103967.0, 105083.0, 109351.0, 109365.0]

[0, 247, 1068, 1090, 1235, 1414, 2848, 3487, 4811, 5865, 7233, 8440, 8730, 9828, 10037, 11849, 18425, 22278, 23649, 34038, 43249, 46549, 50183, 53835, 55997, 65201, 66153, 70862,

In [15]:
df1v3_train = df[df['Pseudonym'].isin(beides_Train)] # Dataframe aus 94tote und 94lebende. Komplettes Train-Set
df1v3_Test = df[df['Pseudonym'].isin(beides_Test)]

In [16]:
df1v3_train = df[df['Pseudonym'].isin(beides_Train)] # Dataframe aus 94tote und 94lebende. Komplettes Train-Set
df1v3_Test = df[df['Pseudonym'].isin(beides_Test)]
df1v3_train.to_csv('model1v3trainunfilled.csv')
df1v3_Test.to_csv('model1v3testunfilled.csv')

Fülle model1v3trainunfilled mit den Werten direkt nach OP, d.h. entweder an Tag 0, falls Wert vorhanden, oder unmittelbar danach.
Mache dasselbe mit model1v3testunfilled

Für jeden Patienten: Nehme einen Ausschnitt von Tag 0 bis 100 aus dem Datensatz als neues Dataframe. Gehe durch alle Zeilen 

In [17]:
def takefreshop(inputDF, pseudo):
    dffilled = inputDF.copy()
    p = dffilled[dffilled['Pseudonym'] == pseudo]
    p.sort_values(by='relatives_datum', ascending=True, inplace=True)
    col = dffilled.columns
    # tmp.to_csv('0.csv')
    naive = pd.DataFrame(columns=col)
    abkuset = set()
    for row in range(0, len(p)):
        for col in range(0, len(p.columns)):
            value = p.iloc[row, col]
            if (np.isnan(value) == False) and (abkuset.__contains__(p.columns[col]) == False): 
                naive.loc[0, p.columns[col]] = value
                abkuset.add(p.columns[col])
                continue
                
    return naive

Model1v3trainunfilled

In [18]:
#Erstelle Dataframe, in der für jeden Patienten nur sein aktuellster Wert pro Feature benutzt wird
inputDf = 'model1v3trainunfilled.csv'
outputDf = 'model1v3train_Filled.csv'
        

df = pd.read_csv(inputDf)
df = df.iloc[:, 1:]
op = df[df['relatives_datum'].between(0,100)]
pseudolist = df['Pseudonym'].unique()
frames = []
i = 0
for name in pseudolist:
    tmp = takefreshop(op, name)
    frames.append(tmp)
    i += 1
result = pd.concat(frames)

In [19]:
print(result.head())

  Pseudonym relatives_datum    CA    CREA   CRP   ERY   EVB  GGT37 GOT37  \
0       0.0             0.0  1.99  0.6801  2.61  3.54  13.7  284.0  86.0   
0      79.0             0.0  1.89  0.5501  0.61  2.67  12.8   38.0   7.0   
0     124.0             0.0  2.11  0.4101  1.01  2.77  18.7  112.0  10.0   
0     247.0             0.0  2.26  0.5501  1.66  2.95  24.2  104.0  88.0   
0     997.0             0.0  2.45  0.8501  0.97  3.32  15.9  167.0  15.0   

   GPT37  ...   IL-6 G.APTT G.QUICK   IRF CA-KORR NRBC-ABS FK-RO IG-ABS  \
0  254.0  ...    NaN    NaN     NaN   NaN     NaN      NaN   NaN    NaN   
0   20.0  ...   8.72   22.9    71.7   1.2    2.24     0.03  5.08    0.1   
0   20.0  ...   99.6   26.1    77.0   0.0  2.3325     0.03   NaN   0.11   
0   33.0  ...  54.47   31.3    80.6  21.2  2.3275     0.05   1.0    NaN   
0   34.0  ...   5.29   29.3    85.9  53.9  2.4325     0.03  9.94   0.13   

  IG-REL   CSA-RO  
0    NaN      NaN  
0    2.8      NaN  
0    1.7      NaN  
0    NaN  30

In [20]:
result.to_csv(outputDf)

Fehlende Werte für Tote werden mit Mittelwert der Toten und entsprechend bei den Lebenden aufgefüllt

In [21]:
df = pd.read_csv('model1v3train_Filled.csv')
df = df.iloc[:, 1:]
lebend = df[df['Pseudonym'].isin(uniquelebende1)]
tot = df[df['Pseudonym'].isin(uniquetote1)]

In [22]:
for i in lebend.columns[lebend.isnull().any(axis=0)]:     #---Applying Only on variables with NaN values
        lebend[i].fillna(lebend[i].mean(),inplace=True)
for i in tot.columns[tot.isnull().any(axis=0)]:     #---Applying Only on variables with NaN values
        tot[i].fillna(tot[i].mean(),inplace=True)

In [23]:
frames = [lebend, tot]
result = pd.concat(frames)
result.to_csv('model1v3train_Filled.csv')

Model1v3testunfilled

In [24]:
inputDf_test = 'model1v3testunfilled.csv'
outputDf_test = 'model1v3test_Filled.csv'
        

df_test = pd.read_csv(inputDf_test)
df_test = df_test.iloc[:, 1:]
op_test = df_test[df_test['relatives_datum'].between(0,100)]
pseudolist_test = df_test['Pseudonym'].unique()
frames_test = []
l = 0
for name in pseudolist_test:
    tmp = takefreshop(op_test, name)
    frames_test.append(tmp)
    l += 1
result_test = pd.concat(frames_test)

In [25]:
print(result_test.head())
result_test.to_csv(outputDf_test)

  Pseudonym relatives_datum    CA    CREA   CRP   ERY        EVB  GGT37 GOT37  \
0  111030.0             0.0  2.13  0.4901  3.54  3.29       18.5   71.0   8.0   
0  112457.0             0.0  2.22  0.4301   0.1  2.93       18.5   54.0  17.0   
0  113054.0             0.0  2.15  0.6701  0.33  2.76  16.299999   46.0  14.0   
0  116887.0             0.0  2.18  0.7901  6.28  3.77       12.7   41.0  12.0   
0  118338.0             0.0  2.14  0.6701  0.92  2.94       13.6  124.0  18.0   

       GPT37  ...  IL-6 G.APTT G.QUICK  IRF CA-KORR NRBC-ABS  FK-RO IG-ABS  \
0        5.0  ...   NaN    NaN     NaN  NaN     NaN      NaN   9.11    NaN   
0  71.900002  ...   NaN    NaN     NaN  NaN     NaN      NaN   4.61    NaN   
0       22.0  ...   NaN    NaN     NaN  NaN     NaN      NaN    NaN    NaN   
0       17.4  ...   NaN    NaN     NaN  NaN     NaN      NaN   14.3    NaN   
0       20.7  ...  1.54   22.6   100.7  0.0     NaN     0.03  18.41   0.08   

  IG-REL      CSA-RO  
0    NaN         NaN 

In [26]:
df = pd.read_csv('model1v3test_Filled.csv')
df = df.iloc[:, 1:]
lebend = df[df['Pseudonym'].isin(uniquelebende2)]
tot = df[df['Pseudonym'].isin(uniquetote2)]

In [27]:
for i in lebend.columns[lebend.isnull().any(axis=0)]:     #---Applying Only on variables with NaN values
        lebend[i].fillna(lebend[i].mean(),inplace=True)
for i in tot.columns[tot.isnull().any(axis=0)]:     #---Applying Only on variables with NaN values
        tot[i].fillna(tot[i].mean(),inplace=True)

frames = [lebend, tot]
result = pd.concat(frames)
result.to_csv('model1v3test_Filled.csv')

In [28]:
inputDf = 'model1v3test_Filled.csv'
statusDf = 'Verstorben.csv'

        #Fügt Status zur InputDf hinzu
ml.addStatusModel1(inputDf, statusDf)

        #Füllt Spaltenweise NaNs mit dem Spaltenmittelwert
df = pd.read_csv(inputDf)
df = df.iloc[:, 2:]
for i in df.columns[df.isnull().any(axis=0)]:     #---Applying Only on variables with NaN values
    df[i].fillna(df[i].mean(),inplace=True)
df.to_csv(inputDf)  

In [29]:
inputDf = 'model1v3train_Filled.csv'
statusDf = 'Verstorben.csv'

        #Fügt Status zur InputDf hinzu
ml.addStatusModel1(inputDf, statusDf)

        #Füllt Spaltenweise NaNs mit dem Spaltenmittelwert
df = pd.read_csv(inputDf)
df = df.iloc[:, 2:]
for i in df.columns[df.isnull().any(axis=0)]:     #---Applying Only on variables with NaN values
    df[i].fillna(df[i].mean(),inplace=True)
df.to_csv(inputDf) 